# Trabalho de Ciência de Dados: Regressão Linear para Predição de Preços de Casas

Neste notebook, você encontrará a estrutura básica para implementar um modelo de Regressão Linear do zero para prever preços de casas. Seu trabalho é preencher as funções com o código necessário para realizar as tarefas descritas.

Lembre-se: Não use bibliotecas como sklearn ou outras que já tenham implementações prontas de regressão linear ou métricas de avaliação.

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'numpy'

## 1. Carregamento e Pré-processamento dos Dados

Comece carregando os dados e realizando qualquer pré-processamento necessário.

In [43]:
def carregar_dados(caminho_arquivo = 'Housing.csv'):
    return pd.read_csv(caminho_arquivo) 

def preprocessar_dados(dados):
    # Colunas categóricas que precisam ser convertidas
    colunas_categoricas = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 
                           'airconditioning', 'prefarea', 'furnishingstatus']
    
    # Separando a coluna alvo (y) e as variáveis independentes (X)
    y = dados['price'].values.astype(float)  # Converta para float explicitamente
    X = dados.drop(columns=['price']).copy()

    # Convertendo as colunas categóricas para variáveis dummy (one-hot encoding)
    X = pd.get_dummies(X, columns=colunas_categoricas, drop_first=True)

    # Normalização dos dados (padronização: média 0, desvio padrão 1) apenas para variáveis numéricas
    colunas_numericas = X.select_dtypes(include=[np.number]).columns
    X_mean = X[colunas_numericas].mean()
    X_std = X[colunas_numericas].std()

    # Evitar divisão por zero no caso de desvio padrão igual a zero
    X_std[X_std == 0] = 1

    # Normalizando os dados
    X[colunas_numericas] = (X[colunas_numericas] - X_mean) / X_std

    # Convertendo X para array numpy e garantindo tipo float
    X = X.values.astype(float)

    return X, y





## 2. Implementação da Regressão Linear

Agora, implemente as funções necessárias para a Regressão Linear usando o Gradiente Descendente.

**Lembre-se:** o objetivo da Regressão Linear é descobrir os parâmetros $w$ (*slope*) e $b$ (*interceptador*), tal que a função linear $y = wx  + b$ tenha o menor erro possível.

In [40]:
class RegressaoLinear:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def funcao_custo(self, X, y, weights, bias):
        m = len(y)
        previsao = np.dot(X, weights) + bias
        erro = previsao - y
        custo = (1 / (2 * m)) * np.sum(erro**2)  # Função de custo
        return custo

    def gradiente_descendente(self, X, y, weights, bias):
        m = len(y)
        previsao = np.dot(X, weights) + bias
        erro = previsao - y

        # gradiente
        dw = (1 / m) * np.dot(X.T, erro)
        db = (1 / m) * np.sum(erro)

        # Converter para float para garantir que não ocorra o erro de casting
        weights = weights.astype(float)
        dw = dw.astype(float)

        weights -= self.learning_rate * dw
        bias -= self.learning_rate * db

        return weights, bias


    def treinar(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.bias = 0

        for i in range(self.num_iterations):
            self.weights, self.bias = self.gradiente_descendente(X, y, self.weights, self.bias)

            if i % 100 == 0:
                custo = self.funcao_custo(X, y, self.weights, self.bias)
                print(f'Iteração {i}, Custo: {custo}')

    def prever(self, X):
        return np.dot(X, self.weights) + self.bias

## 3. Métricas de Avaliação

Implemente as funções para calcular as métricas de avaliação do modelo.

In [41]:
def calcular_rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))  # Corrigido: Agora eleva o erro ao quadrado

def calcular_mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def calcular_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## Função para dividir treino e teste
def dividir_treino_teste(X, y, test_size=0.2):
    n_total = X.shape[0]
    n_teste = int(n_total * test_size)

    # Embaralhar os dados
    indices = np.random.permutation(n_total)
    indices_teste = indices[:n_teste]
    indices_treino = indices[n_teste:]

    X_treino, X_teste = X[indices_treino], X[indices_teste]
    y_treino, y_teste = y[indices_treino], y[indices_teste]

    return X_treino, X_teste, y_treino, y_teste

## 4. Treinamento e Avaliação do Modelo

Use as funções implementadas para treinar o modelo e avaliar seu desempenho.

In [44]:
# Carregue e pré-processe os dados
dados = carregar_dados('Housing.csv')
X, y = preprocessar_dados(dados)

# Divida os dados em conjuntos de treino e teste
X_treino, X_teste, y_treino, y_teste = dividir_treino_teste(X, y, test_size=0.2)

# Crie e treine o modelo
modelo = RegressaoLinear()
modelo.treinar(X_treino, y_treino)

# Faça previsões
y_pred = modelo.prever(X_teste)

# Calcule e imprima as métricas
rmse = calcular_rmse(y_teste, y_pred)
mae = calcular_mae(y_teste, y_pred)
mape = calcular_mape(y_teste, y_pred)

print(f'RMSE (Teste): {rmse:.5f}')
print(f'MAE (Teste): {mae:.5f}')
print(f'MAPE (Teste): {mape:.5f}')


UFuncTypeError: Cannot cast ufunc 'subtract' output from dtype('O') to dtype('float64') with casting rule 'same_kind'

## 5. Visualização dos Resultados

Crie visualizações para ajudar a interpretar os resultados do seu modelo.

In [ ]:
def plotar_resultados(y_teste, y_pred):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_teste, y_pred, color='blue', label='Valores Previstos')
    plt.plot([y_teste.min(), y_teste.max()], [y_teste.min(), y_teste.max()], color='red', lw=2, label='Linha Ideal')
    plt.xlabel('Valores Reais')
    plt.ylabel('Valores Previstos')
    plt.title('Valores Reais vs. Valores Previstos')
    plt.legend()
    plt.show()

plotar_resultados(y_teste, y_pred)